# FID with GAN generation

In [2]:
from __future__ import print_function

import argparse
import math
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from torchfusion.gan.applications import DCGANDiscriminator

from data_loader import MarioDataset
from models.custom import Generator

import csv

from image_gen.asset_map import get_asset_map
from image_gen.fixer import PipeFixer
from image_gen.image_gen import GameImageGenerator
from tqdm import tqdm

from get_level import GetLevel as getLevel
from scipy.linalg import sqrtm

/home/ycv5080/miniconda3/envs/cs449_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ycv5080/miniconda3/envs/cs449_project/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/ycv5080/miniconda3/envs/cs449_project/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


ModuleNotFoundError: No module named 'torch.ao.quantization'

### Functions for FID

In [4]:
torch.cuda.is_available()

True

In [3]:
# Function to preprocess the matrices
def preprocess_matrices(matrices):
    # Normalize values to the range [0, 255]
    normalized_matrices = (matrices - np.min(matrices)) / (np.max(matrices) - np.min(matrices))
    normalized_matrices = normalized_matrices * 255
    return normalized_matrices.astype(np.uint8)

# Function to compute mean and covariance of features
def compute_statistics(matrices):
    # Flatten matrices into vectors
    flattened_matrices = matrices.reshape((matrices.shape[0], -1))
    # Compute mean and covariance
    mean = np.mean(flattened_matrices, axis=0)
    covariance = np.cov(flattened_matrices, rowvar=False)

    return mean, covariance

# Function to compute Fréchet distance
def compute_frechet_distance(real_mean, real_cov, generated_mean, generated_cov):
    epsilon = 1e-6  # Small constant to avoid numerical instability
    sqrt_cov_product = sqrtm(real_cov.dot(generated_cov))
    fid_score = np.linalg.norm(real_mean - generated_mean) + np.trace(real_cov + generated_cov - 2 * sqrt_cov_product)

    return fid_score

## Batches for Real samples

In [4]:
org_data = MarioDataset()
ref_idx = torch.randperm(len(org_data))
prev_frame, curr_frame = (org_data[:].prev_frame, org_data[:].curr_frame)
complete_frame = torch.cat((prev_frame,curr_frame),dim=3)
complete_frame = torch.argmax(complete_frame, dim = 1)
#complete_frame = torch.tensor(complete_frame,dtype=torch.uint8)

In [5]:
# Preprocess real data
complete_frame_np = complete_frame.detach().numpy()
real_matrices = preprocess_matrices(complete_frame_np)
# Compute statistics for real matrices
real_mean, real_cov = compute_statistics(real_matrices)


### Batches for generated data

In [6]:
conditional_channels = [0,1,6,7]
def Gen_sample(conditional_channels,ini_data=120):
    dataset = MarioDataset()
    netG = Generator(
            latent_size=(len(conditional_channels) + 1, 14, 14), out_size=(13, 32, 32)
        )
    netG.load_state_dict(torch.load("./trained_models/netG_epoch_300000_0_32.pth"))
        # 300000
    mario_map = get_asset_map(game="mario")
    gen = GameImageGenerator(asset_map=mario_map)
    prev_frame, curr_frame = dataset[[ini_data]]
    fixer = PipeFixer()

    level_gen = getLevel(netG, gen, fixer, prev_frame, curr_frame, conditional_channels)
    var = 1
    #noise = np.rand((1, 1, 14, 14)).normal_(0, var)
    noise = np.random.normal(0,var,size=(14,14))
    level = level_gen.generate_frames(noise, var=var, frame_count=1) # generated matrix without padded
    padded = torch.zeros(32,32)
    padded[9:-9,2:-2] = torch.from_numpy(level)
    level = padded
    #level_finalize = torch.zeros(1,32,32)
    #level_finalize[0,:,:] = level
    #level = level_finalize
    return level
    # this is just for visualization
    #level_gen.gen.save_gen_level(img_name="test_fuc_gen")

In [20]:
# number of generated samples 
n_gen = 1000

# generate samples and stack together
level_gen = torch.zeros(n_gen,32,32)
for i in range(n_gen):
    level = Gen_sample(conditional_channels,ini_data=120)
    level_gen[i,:,:] = level


In [1]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

In [21]:
# Preprocess generated data
generated_frame_np = level_gen.detach().numpy()
generated_matrices = preprocess_matrices(generated_frame_np)
# Compute statistics for real matrices
gen_mean, gen_cov = compute_statistics(generated_matrices)

In [22]:
# Compute Fréchet distance
fid_score = compute_frechet_distance(real_mean, real_cov, gen_mean, gen_cov)
print("FID score:", fid_score)

FID score: (136664.82718125774-1.1180391749244107e-05j)
